In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import pandas as pd
import os

In [2]:
load_dotenv()

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [3]:
import json

with open("../../out/prompting_ners_gpt_4.1_mini.json", "r", encoding="utf-8") as fr:
    prompting_ners = json.load(fr)

In [4]:
import ast
    
data = pd.read_csv("../../data/DicatJuanRana_w_sentences.csv", sep=";", encoding="utf-8", converters={"sentences": ast.literal_eval})

In [15]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["text", "prev_context", "post_context"],
    template="""
        Actúa como un agente de procesamiento de lenguaje natural cuyo objetivo es aclarar referencias ambiguas en un texto, especialmente aquellas relacionadas con:

         - Sujeto omitido o implícito (ej. "fue", "llegó", "respondió")

         - Complemento directo (ej. "la comedia", "la llamó", "lo rechazaron")

         - Complemento indirecto (ej. "le dijo", "le enviaron")

        Tu tarea es sustituir estas referencias por el nombre o entidad concreta a la que se refieren, usando el contexto de hasta dos frases anteriores y dos posteriores.

        Reescribe únicamente las frases necesarias para que queden claras, manteniendo la coherencia sintáctica. Las frases que ya están completas y claras deben dejarse igual.
        
        No digas que la frase es ambigua, simplemente realiza las sustituciones necesarias.
        
        **EJEMPLO Complemento Directo**

        Entrada:
        La comedia fue muy bien. La comedia era Constantino.

        Salida:
        Constantino fue muy bien. La comedia era Constantino.
        
        **EJEMPLO Sujeto Omitido**

        Entrada:
        Ayer llegaron todos menos Ana. Fue quien organizó el evento. Nadie se lo esperaba.

        Salida:
        Ayer llegaron todos menos Ana. Ana organizó el evento. Nadie se lo esperaba.
        
        Contexto Previo:
        {prev_context}
        
        Contexto Posterior:
        {post_context}
        
        Texto:
        {text}
    """,
)

In [7]:
texto_to_clarify = data[data["year"].astype(int) == 1653]["sentences"].values[0]
texto_to_clarify

['En una carta, fechada en Madrid el 9 de junio de este año, Felipe IV daba cuenta a doña Luisa Enríquez Manrique de una comedia representada en el Coliseo del Buen Retiro en la segunda quincena de mayo:  ',
 '"Muy buena ha sido la comedia, y vuestro amigo \'Juan Rana\' a cumplido famosamente con sus obligaciones;  ',
 'bien creo que si le hubiérades visto, se atreviera la risa a inquietar la función.  ',
 'La gente moza se ha entretenido harto" —la comedia representada fue Andrómeda y Perseo, de Calderón, en la que Cosme Pérez representó el papel de "Bato"—;  ',
 'entre la documentación relativa a las fiestas del Corpus de Madrid consta una lista de los gastos de las fiestas del Corpus de este año, en la que se indica que Cosme Pérez recibió 1.000 rs.;  ',
 'consta la respuesta, transmitida el 22 de noviembre por el Conde de Altamira, a la solicitud de "Cosme Pérez, alias Juan Rana" realizada el 18 de dicho mes, referente al "paso de una ración ordinaria que goza por la despensa de la

In [20]:
from tqdm.auto import tqdm

sentences_of_context = 3 # Number of sentences to consider as context for each sentence
all_clarified_sentences = []

for _, year in tqdm(data.iterrows(), total=len(data)):

    prev_context = []
    post_context = []

    result = []
    texto_to_clarify = year["sentences"]

    for i, sentence in tqdm(enumerate(texto_to_clarify), total=len(texto_to_clarify), leave=False):
        prev_context = result[-2:]
        post_context = texto_to_clarify[i + 1:i + 1 + sentences_of_context]
        
        prev_context = " ".join(prev_context)
        post_context = " ".join(post_context)
        
        response = llm.invoke(
            prompt.format(
                text=sentence,
                prev_context=prev_context,
                post_context=post_context
            )
        ).content
        
        result.append(response)
    
    all_clarified_sentences.append(result)

  0%|          | 0/38 [00:00<?, ?it/s]

  0%|          | 0/89 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
import rich as rc

print("Texto original:")
rc.print(" ".join(texto_to_clarify))

print("\nTexto corregido:")
rc.print(" ".join(result))

Texto original:


En una carta, fechada en Madrid el 9 de junio de este año, Felipe IV daba cuenta a doña Luisa Enríquez Manrique de 
una comedia representada en el Coliseo del Buen Retiro en la segunda quincena de mayo:   "Muy buena ha sido la 
comedia, y vuestro amigo 'Juan Rana' a cumplido famosamente con sus obligaciones;   bien creo que si le hubiérades 
visto, se atreviera la risa a inquietar la función.   La gente moza se ha entretenido harto" —la comedia 
representada fue Andrómeda y Perseo, de Calderón, en la que Cosme Pérez representó el papel de "Bato"—;   entre la 
documentación relativa a las fiestas del Corpus de Madrid consta una lista de los gastos de las fiestas del Corpus 
de este año, en la que se indica que Cosme Pérez recibió 1.000 rs.;   consta la respuesta, transmitida el 22 de 
noviembre por el Conde de Altamira, a la solicitud de "Cosme Pérez, alias Juan Rana" realizada el 18 de dicho mes, 
referente al "paso de una ración ordinaria que goza por la despensa de la casa de la Reyna nuestra señora, para una
hija suya [Francisca María Pérez], con calidad que sea desde luego y que la susodicha no la pueda gozar si hubiese 
de andar en la farsa".


Texto corregido:


En una carta, fechada en Madrid el 9 de junio de este año, Felipe IV daba cuenta a doña Luisa Enríquez Manrique de 
la comedia Andrómeda y Perseo, de Calderón, representada en el Coliseo del Buen Retiro en la segunda quincena de 
mayo: Muy buena ha sido la comedia Andrómeda y Perseo, y vuestro amigo 'Juan Rana' ha cumplido famosamente con sus 
obligaciones; Bien, creo que si hubierais visto a Cosme Pérez, alias Juan Rana, se atrevería la risa a inquietar la
función. La gente moza se ha entretenido harto" —la comedia representada fue Andrómeda y Perseo, de Calderón, en la
que Cosme Pérez representó el papel de "Bato"—; Entre la documentación relativa a las fiestas del Corpus de Madrid 
consta una lista de los gastos de las fiestas del Corpus de este año, en la que se indica que Cosme Pérez recibió 
1.000 rs.; Consta la respuesta, transmitida el 22 de noviembre por el Conde de Altamira, a la solicitud de "Cosme 
Pérez, alias Juan Rana" realizada el 18 de dicho mes, referente al "paso de una ración ordinaria que goza por la 
despensa de la casa de la Reyna nuestra señora, para una hija suya, Francisca María Pérez, con calidad que sea 
desde luego y que Francisca María Pérez no pueda gozar la ración si hubiese de andar en la farsa".

In [ ]:
data["clarified_sentences"] = all_clarified_sentences
data.to_csv("../../data/DicatJuanRana_w_clarified_sentences.csv", sep=";", encoding="utf-8", index=False)

In [29]:
data[data["year"].astype(int) == 1653]["clarified_sentences"].values[0]

['En una carta, fechada en Madrid el 9 de junio de este año, Felipe IV daba cuenta a doña Luisa Enríquez Manrique de la comedia Andrómeda y Perseo, de Calderón, representada en el Coliseo del Buen Retiro en la segunda quincena de mayo:',
 "Muy buena ha sido la comedia Andrómeda y Perseo, y vuestro amigo 'Juan Rana' ha cumplido famosamente con sus obligaciones;",
 'Bien, creo que si hubierais visto a Cosme Pérez, alias Juan Rana, se atrevería la risa a inquietar la función.',
 'La gente moza se ha entretenido harto" —la comedia representada Andrómeda y Perseo, de Calderón, fue Andrómeda y Perseo, en la que Cosme Pérez representó el papel de "Bato"—;',
 'entre la documentación relativa a las fiestas del Corpus de Madrid consta una lista de los gastos de las fiestas del Corpus de este año, en la que se indica que Cosme Pérez recibió 1.000 rs.;',
 'Consta la respuesta, transmitida el 22 de noviembre por el Conde de Altamira, a la solicitud de "Cosme Pérez, alias Juan Rana" realizada el 18 

In [30]:
for _, row in data.iterrows():
    if len(row["clarified_sentences"]) != len(row["sentences"]):
        print(f"Error in year {row['year']}: {len(row['clarified_sentences'])} clarified sentences vs {len(row['sentences'])} original sentences.")
        break